<a href="https://colab.research.google.com/github/jexiao8211/CSDS312-FINAL-PROJECT/blob/main/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvcc -V && which nvcc

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
/usr/local/cuda/bin/nvcc


In [4]:
!nvidia-smi

Mon May  1 03:12:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 385, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 385 (delta 86), reused 51 (delta 51), pack-reused 269
Receiving objects: 100% (385/385), 105.74 KiB | 8.81 MiB/s, done.
Resolving deltas: 100% (188/188), done.


In [5]:
!python3 -m pip install pynvml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 852.0 kB/s eta 0:00:00


In [6]:
!python3 rapidsai-csp-utils/colab/env-check.py

***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS via pip!  Please stand by, should be quick...
***********************************************************************



In [4]:
!python rapidsai-csp-utils/colab/pip-install.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pynvml-11.5.0-py3-none-any.whl (53 kB)
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.6/496.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.4/428.4 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [5]:
import cudf
cudf.__version__

'23.04.01'

In [6]:
import cuml
cuml.__version__

'23.04.01'

In [35]:
import cudf
import numpy as np
import pandas as pd
import pickle

from cuml.ensemble import RandomForestClassifier as curfc
from cuml.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier as skrfc
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [36]:
# Replace 'your_file.csv' with the name of your CSV file
file_path = 'final_data.csv'

# Read the CSV file using pandas
df = pd.read_csv(file_path)

# Drop unnamed columns
unnamed_columns = df.columns[df.columns.str.contains('^Unnamed')]
df = df.drop(columns=unnamed_columns)

df = df.dropna()

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18584 entries, 0 to 19560
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   home_matches_played          18584 non-null  int64  
 1   home_wins                    18584 non-null  int64  
 2   home_win_rate                18584 non-null  float64
 3   home_buildUpPlaySpeed        18584 non-null  float64
 4   home_buildUpPlayDribbling    18584 non-null  float64
 5   home_buildUpPlayPassing      18584 non-null  float64
 6   home_chanceCreationPassing   18584 non-null  float64
 7   home_chanceCreationCrossing  18584 non-null  float64
 8   home_chanceCreationShooting  18584 non-null  float64
 9   home_defencePressure         18584 non-null  float64
 10  home_defenceAggression       18584 non-null  float64
 11  home_defenceTeamWidth        18584 non-null  float64
 12  away_matches_played          18584 non-null  int64  
 13  away_wins       

In [38]:
# n_samples = 2*17
n_samples = 2**12
n_features = 399
n_info = 300
data_type = np.float32

In [37]:
predict_columns = ['home_win', 'draw', 'away_win']
feature_columns = [col for col in df.columns if col not in predict_columns]

X = df[feature_columns]
y = pd.DataFrame(np.argmax(df[predict_columns].values, axis=1))

### Local Host

In [39]:
%%time
X,y = make_classification(n_samples=n_samples,
                          n_features=n_features,
                          n_informative=n_info,
                          random_state=123, n_classes=2)

X = pd.DataFrame(X.astype(data_type))
# cuML Random Forest Classifier requires the labels to be integers
y = pd.Series(y.astype(np.int32))

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2,
                                                    random_state=0)

CPU times: user 151 ms, sys: 95.7 ms, total: 247 ms
Wall time: 300 ms


### GPU

In [40]:
%%time
X_cudf_train = cudf.DataFrame.from_pandas(X_train)
X_cudf_test = cudf.DataFrame.from_pandas(X_test)

y_cudf_train = cudf.Series(y_train.values)

CPU times: user 310 ms, sys: 0 ns, total: 310 ms
Wall time: 314 ms


# Random Forest

### Fit Model

In [41]:
# fit sklearn RandomForest
%%time
sk_model = skrfc(n_estimators=40,
                 max_depth=16,
                 max_features=1.0,
                 random_state=10)

sk_model.fit(X_train, y_train)

CPU times: user 42.7 s, sys: 44.7 ms, total: 42.8 s
Wall time: 55.4 s


RandomForestClassifier(max_depth=16, max_features=1.0, n_estimators=40,
                       random_state=10)

In [42]:
# evaluate
%%time
sk_predict = sk_model.predict(X_test)
sk_acc = accuracy_score(y_test, sk_predict)

CPU times: user 1.08 s, sys: 668 ms, total: 1.75 s
Wall time: 2.99 s


In [43]:
# cuML RandomForest
%%time
cuml_model = curfc(n_estimators=40,
                   max_depth=16,
                   max_features=1.0,
                   random_state=10)

cuml_model.fit(X_cudf_train, y_cudf_train)

/usr/local/lib/python3.10/site-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


CPU times: user 3.64 s, sys: 166 ms, total: 3.81 s
Wall time: 4.83 s


RandomForestClassifier()

In [44]:
# evaluate
%%time
fil_preds_orig = cuml_model.predict(X_cudf_test)

fil_acc_orig = accuracy_score(y_test.to_numpy(), fil_preds_orig)

CPU times: user 10.8 s, sys: 100 ms, total: 10.9 s
Wall time: 11 s


### Pickle cuML random forest classification model

In [45]:
filename = 'cuml_random_forest_model.sav'
# save the trained cuml model into a file
pickle.dump(cuml_model, open(filename, 'wb'))
# delete the previous model to ensure that there is no leakage of pointers.
# this is not strictly necessary but just included here for demo purposes.
del cuml_model
# load the previously saved cuml model from a file
pickled_cuml_model = pickle.load(open(filename, 'rb'))

In [47]:
# predict using the pickled model
%%time
pred_after_pickling = pickled_cuml_model.predict(X_cudf_test)

fil_acc_after_pickling = accuracy_score(y_test.to_numpy(), pred_after_pickling)

CPU times: user 321 ms, sys: 52.5 ms, total: 374 ms
Wall time: 729 ms


### Compare Results

In [48]:
print("CUML accuracy of the RF model before pickling: %s" % fil_acc_orig)
print("CUML accuracy of the RF model after pickling: %s" % fil_acc_after_pickling)

CUML accuracy of the RF model before pickling: 0.7512195110321045
CUML accuracy of the RF model after pickling: 0.7512195110321045


In [49]:
print("SKL accuracy: %s" % sk_acc)
print("CUML accuracy before pickling: %s" % fil_acc_orig)

SKL accuracy: 0.6926829218864441
CUML accuracy before pickling: 0.7512195110321045


# Linear Regression

In [50]:
import cudf
from cuml import make_regression, train_test_split
from cuml.linear_model import LinearRegression as cuLinearRegression
from cuml.metrics.regression import r2_score
from sklearn.linear_model import LinearRegression as skLinearRegression

In [51]:
# define params
n_samples = 2**20 #If you are running on a GPU with less than 16GB RAM, please change to 2**19 or you could run out of memory
n_features = 399

random_state = 23

In [59]:
%%time
X, y = make_regression(n_samples=n_samples, n_features=n_features, random_state=random_state)

X = cudf.DataFrame(X)
y = cudf.DataFrame(y)[0]

X_cudf, X_cudf_test, y_cudf, y_cudf_test = train_test_split(X, y, test_size = 0.2, random_state=random_state)

CPU times: user 1.16 s, sys: 130 ms, total: 1.29 s
Wall time: 1.93 s


In [60]:
# Copy dataset from GPU memory to host memory.
# This is done to later compare CPU and GPU results.
X_train = X_cudf.to_pandas()
X_test = X_cudf_test.to_pandas()
y_train = y_cudf.to_pandas()
y_test = y_cudf_test.to_pandas()

## Scikit-learn Model

### Fit Model

In [61]:
%%time
ols_sk = skLinearRegression(fit_intercept=True,
                            n_jobs=-1)

ols_sk.fit(X_train, y_train)

CPU times: user 39.8 s, sys: 7.51 s, total: 47.3 s
Wall time: 37.8 s


LinearRegression(n_jobs=-1)

### Predict

In [62]:
%%time
predict_sk = ols_sk.predict(X_test)

CPU times: user 109 ms, sys: 928 µs, total: 109 ms
Wall time: 85.1 ms


### Evaluate

In [74]:
%%time
r2_score_sk = r2_score(y_test, predict_sk)
acc_sk_linreg = accuracy_score(y_test, sk_predict)

CPU times: user 9.75 ms, sys: 0 ns, total: 9.75 ms
Wall time: 9.82 ms


## cuML Model

### Fit Model

In [67]:
%%time
ols_cuml = cuLinearRegression(fit_intercept=True,
                              algorithm='eig')

ols_cuml.fit(X_cudf, y_cudf)

CPU times: user 218 ms, sys: 4.48 ms, total: 223 ms
Wall time: 255 ms


LinearRegression()

### Predict

In [68]:
%%time
predict_cuml = ols_cuml.predict(X_cudf_test)

CPU times: user 97.1 ms, sys: 3.14 ms, total: 100 ms
Wall time: 158 ms


### Evaluate

In [71]:
%%time
r2_score_cuml = r2_score(y_cudf_test, predict_cuml)
acc_origin_linreg = accuracy_score(y_cudf_test.to_numpy(), predict_cuml)

CPU times: user 1.78 s, sys: 16.7 ms, total: 1.8 s
Wall time: 2.84 s


## Pickle Linear Regression Model

In [72]:
filename = 'cuml_linear_regression_model.sav'
# save the trained cuml model into a file
pickle.dump(ols_cuml, open(filename, 'wb'))
# delete the previous model to ensure that there is no leakage of pointers.
# this is not strictly necessary but just included here for demo purposes.
del ols_cuml
# load the previously saved cuml model from a file
pickled_cuml_model = pickle.load(open(filename, 'rb'))

In [73]:
# predict using the pickled model
%%time
pred_after_pickling = pickled_cuml_model.predict(X_cudf_test)

fil_acc_after_pickling = accuracy_score(y_test.to_numpy(), pred_after_pickling)

CPU times: user 55.3 ms, sys: 5.04 ms, total: 60.4 ms
Wall time: 62 ms


### Compare Results

In [75]:
print("CUML accuracy of the RF model before pickling: %s" % acc_origin_linreg)
print("CUML accuracy of the RF model after pickling: %s" % fil_acc_after_pickling)

CUML accuracy of the RF model before pickling: 0.9966192245483398
CUML accuracy of the RF model after pickling: 0.9966192245483398


In [76]:
print("SKL accuracy: %s" % acc_sk_linreg)
print("CUML accuracy before pickling: %s" % acc_origin_linreg)

SKL accuracy: 0.002439024392515421
CUML accuracy before pickling: 0.9966192245483398


## Compare Results

In [77]:
print("R^2 score (SKL):  %s" % r2_score_sk)
print("R^2 score (cuML): %s" % r2_score_cuml)

R^2 score (SKL):  1.0
R^2 score (cuML): 1.0


# SVC

In [78]:
from cuml.svm import SVC as SVC_gpu
clf_svc = SVC_gpu(kernel='poly', degree=2, gamma='auto', C=1)
clf_svc.fit(X_train, y_train)

In [79]:
# define params
max_iter = 300
n_clusters = 3
random_state = 42
init = 'scalable-k-means++'

In [82]:
host_data = X.to_pandas()
host_labels = y.to_pandas()

## Scikit-learn Model

## cuML Model

In [ ]:
kmeans_cuml = cuKMeans(
    init="k-means||",
    n_clusters=n_clusters,
    random_state=random_state
)

%timeit kmeans_cuml.fit(X)